In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import extmath
from sklearn.decomposition import PCA, NMF
from sklearn.feature_selection import SelectKBest, chi2, GenericUnivariateSelect, mutual_info_classif,f_classif

## Dataset

In [2]:
def read_csv(filename):
    dataframe = pd.read_csv(filename, header=None)
#     dataframe = dataframe.sample(frac=1)
    # takes care of nan values

    dataframe=dataframe[:10000].fillna(0)
    
    dataset = dataframe.values    
    header_overall=dataset[0,1:]
    header_x=dataset[0,45:]
    X = dataset[1:,45:]
    Y = dataset[1:,1:45]
    #ds.to_csv('compiled_predicted.csv')
    return header_overall,header_x,X,Y

In [3]:
filename="/home/rahulsuresh/Downloads/mik_dataset/Model_Sample_Set.csv"
header_overall,header_x,x,y=read_csv(filename)

/home/rahulsuresh/python-environments/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,2

## Problem with MIK dataset
* Unable to open with libreoffice - so hard but not impossible to visualize
* Strings
* nan

In [4]:
x[0]

array(['10-NOV-18 12.00.00.000000000 AM', '2018280', '1000171103093', 'N',
       '16-FEB-15 12.00.00.000000000 AM',
       '01-OCT-18 12.00.00.000000000 AM', '221', '8', '123.42', '82',
       '40', '46', '1211071660155', '93.37', '5', '58', '38', '1', '9',
       '38', '1', '9', '38', '1', '9', '0', '0', '0', '93.37', '5', '58',
       '14.59', '13', '21.19', '65.09', '7.09', '218', 'store_only', '38',
       '1', '9', '11.1919', '4', '2', '36', '0', '38', '1', '9',
       '11.1919', '4', '2', '36', '0', '38', '1', '9', '11.1919', '4',
       '2', '36', '0', '0', '0', '0', '0', '0', '0', '0', '0', 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, '1211071660155',
       '3', '1', '3', '596', '0', '0', '0', 0, '0', '0', '0', 0, '2', '1',
       '2', '40', '0.75', '1', '1', '695', '0', '0', '0', 0, '0', '0',
       '0', 0, '0', '0', '0', 0, '0', '0', '0', 0, '0', '0', '0', 0, '0',
       '0', '0', 0, '0', '0', '0', 0, '0', '0', '0', 0, '0', '0', '0', 0,
       '0', '0'

In [5]:
print(x.shape)
print(header_x.shape)
to_delete=[44,45,46,47,48,49,56,81,134,850,851,852,853,854,856,857,858,875]
to_delete2 = [ele - 44 for ele in to_delete]

(9999, 838)
(838,)


### Removing useless columns - also takes care of strings

In [6]:
def delete_columns(x,to_delete_list):
    for ele in to_delete_list:
        x=np.delete(x, np.s_[ele], axis=1)
        to_delete_list[:] = [e - 1 for e in to_delete_list]
    return x

In [7]:
def delete_columns_vector(a,index):
    a=np.delete(a, index)
    return a

In [8]:
x=delete_columns(x,to_delete2)

In [11]:
to_delete2 = [ele - 44 for ele in to_delete]
header_x=delete_columns_vector(header_x,to_delete2)

In [12]:
def convert_dtype_linear_shape_y(x,y):
    x=x.astype('float')
    y=y.astype('float')    
    y_new=[]
    for row in y:
        y_new.append(np.argmax(row))
    return x,np.array(y_new)

In [13]:
x,_=convert_dtype_linear_shape_y(x,y)

In [14]:
y=y.astype('float')

In [65]:
y[4]

array([1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

## Preparing Data

Task 1 -> generate a sample multilabel dataset

In [15]:
# from sklearn.datasets import make_multilabel_classification

# x,y=make_multilabel_classification(n_samples=8000, n_features=80, 
#                                              n_classes=10, n_labels=4, length=100, 
#                                              allow_unlabeled=True, sparse=False, return_indicator="dense", 
#                                              return_distributions=False, random_state=None)

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = .2,random_state=12)

In [17]:
x_train.shape

(7999, 820)

### Applying Oversampling

In [31]:
import numpy as np

def label_counter(y):
    return np.unique(y,return_counts=True)

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer

def to_multilabel_csr(y):   
    multilabel_binarizer = MultiLabelBinarizer()
    multilabel_binarizer.fit(y)
    y = multilabel_binarizer.transform(y)
    return y

In [37]:
def get_data(x,y,acc_list,labels):
    x_final=[]
    y_final=[]
    for i in acc_list:
        search_label=labels[i]
        for j in range(len(y)):
            if(y[j]==search_label):
                x_final.append(x[j])
                y_final.append(y[j])
    return x_final,y_final    

In [38]:
from imblearn.over_sampling import SMOTE
from scipy.sparse import csr_matrix
# import warnings
# warnings.filterwarnings(action='ignore')

# Problem : Encoding sample has a single unique appearance

def smote_sampler(x_train,yt,lp):
    labels,vals=label_counter(yt)

    ratio_list=[]
    delete_list=[]
    
    for i in range(len(np.unique(yt))):
        if (vals[i]>6):
            ratio_list.append(i)
        else:
            delete_list.append(i)

    x_train_sm,yt_sm=get_data(x_train,yt,ratio_list,labels)

    x_train_rem,yt_rem=get_data(x_train,yt,delete_list,labels)
            
    sm = SMOTE(random_state=42)

    x_res, y_res = sm.fit_sample(x_train_sm, yt_sm)

    y_res = lp.inverse_transform(y_res)
        
    yt_rem = lp.inverse_transform(yt_rem)
    
    arr=np.concatenate((y_res.rows, yt_rem.rows), axis=0)
    
    y_res=to_multilabel_csr(arr)
    
    return np.concatenate((x_res, x_train_rem), axis=0),y_res

In [39]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from scipy.sparse import csr_matrix
# import warnings
# warnings.filterwarnings(action='ignore')

# Problem : Encoding sample has a single unique appearance

def smoteenn_sampler(x_train,yt,lp):
    labels,vals=label_counter(yt)

    ratio_list=[]
    delete_list=[]
    
    for i in range(len(np.unique(yt))):
        if (vals[i]>6):
            ratio_list.append(i)
        else:
            delete_list.append(i)

    x_train_sm,yt_sm=get_data(x_train,yt,ratio_list,labels)

    x_train_rem,yt_rem=get_data(x_train,yt,delete_list,labels)
            
    sm = SMOTEENN(random_state=42)

    x_res, y_res = sm.fit_sample(x_train_sm, yt_sm)

    y_res = lp.inverse_transform(y_res)
        
    yt_rem = lp.inverse_transform(yt_rem)
    
    arr=np.concatenate((y_res.rows, yt_rem.rows), axis=0)
    
    y_res=to_multilabel_csr(arr)
    
    return np.concatenate((x_res, x_train_rem), axis=0),y_res

In [40]:
from skmultilearn.problem_transform import LabelPowerset

lp = LabelPowerset()

# Applies the above stated multi-label (ML) to multi-class (MC) transformation.
yt = lp.transform(y_train)

#### SMOTE

In [41]:
x_res,y_res=smote_sampler(x_train,yt,lp)

#### SMOTEENN

In [42]:
x_res2,y_res2=smoteenn_sampler(x_train,yt,lp)

## Adapted Algorithms

### Summary


| Algorithm        | Data Oversampling technique | Accuracy(do x 100 for %) | Hamming Loss |
| :-------------: |:--------:|:-----:|:-----:|
| MultiLearn KNN (MLKNN)     | None | 0.004 | 0.05646590909090909 |
| Binary Relevance a KNN (BRaKNN)     | None | 0.001 |  0.055625 |
| Binary Relevance b KNN (BRbKNN)     | None | 0.0 |  0.10853409090909091 |
| Multi-label ARAM (MLARAM)     | None | 0.0 |  0.10853409090909091 |
| Decision Tree     | None | 0.0 |  0.10853409090909091 |
| Extra Tree     | None | 0.0 |  0.10853409090909091 |
| Extra Tree Ensemble     | None | 0.0 |  0.10853409090909091 |
| Random Forest     | None | 0.0 |  0.10853409090909091 |
| | | | |
| MultiLearn KNN (MLKNN)     | SMOTE | 0.0045 | 0.0655340909090909 |
| Binary Relevance a KNN (BRaKNN)     | SMOTE | 0.004 |  0.06780681818181818 |
| Binary Relevance b KNN (BRbKNN)     | SMOTE | 0.0005 |  0.09689772727272727 |
| Multi-label ARAM (MLARAM)     | SMOTE | 0.0005 |  0.09689772727272727 |
| Decision Tree     | SMOTE | 0.0005 |  0.10853409090909091 |
| Extra Tree     | SMOTE | 0.0005 |  0.10853409090909091 |
| Extra Tree Ensemble     | SMOTE | 0.0005 |  0.10853409090909091 |
| Random Forest     | SMOTE | 0.0005 |  0.10853409090909091 |
| | | | |
| MultiLearn KNN (MLKNN)     | SMOTEENN | 0.0045 | 0.06894318181818182 |
| Binary Relevance a KNN (BRaKNN)     | SMOTEENN | 0.004 |  0.07055681818181818 |
| Binary Relevance b KNN (BRbKNN)     | SMOTEENN | 0.0005 |   0.09932954545454545 |
| Multi-label ARAM (MLARAM)     | SMOTEENN | 0.0005 |   0.09932954545454545 |
| Decision Tree     | SMOTEENN | 0.0005 |   0.09932954545454545 |
| Extra Tree     | SMOTEENN | 0.0005 |   0.09932954545454545 |
| Extra Tree Ensemble     | SMOTEENN | 0.0005 |   0.09932954545454545 |
| Random Forest     | SMOTEENN | 0.0005 |   0.09932954545454545 |

### MultiLearn KNN (MLKNN)

In [55]:
from skmultilearn.adapt import MLkNN
from sklearn.metrics import accuracy_score,hamming_loss

classifier = MLkNN(k=10)

# train
classifier.fit(x_res2, y_res2)

# predict
predictions = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.0045 Hamming Loss: 0.06894318181818182


### Binary Relevance a KNN (BRaKNN)

In [56]:
from skmultilearn.adapt import BRkNNaClassifier
from sklearn.metrics import accuracy_score,hamming_loss

classifier = BRkNNaClassifier(k=10)

# train
classifier.fit(x_res2, y_res2)

# predict
predictions = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.004 Hamming Loss: 0.07055681818181818


### Binary Relevance b KNN (BRbKNN)

In [57]:
from skmultilearn.adapt import BRkNNbClassifier
from sklearn.metrics import accuracy_score,hamming_loss

classifier = BRkNNbClassifier(k=10)

# train
classifier.fit(x_res2, y_res2)

# predict
predictions = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### Multi-label ARAM (MLARAM)

In [58]:
from skmultilearn.adapt import MLARAM
from sklearn.metrics import accuracy_score,hamming_loss

classifier = MLARAM(threshold=0.05, vigilance=0.95)
classifier.fit(x_res2, y_res2)
prediction = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### Twin Multi Label Support Vector Machines (MLTSVM)   -> Not working; No support from sk multilearn

In [22]:
# from skmultilearn.adapt import MLTSVM
# from sklearn.metrics import accuracy_score,hamming_loss

# classifier = MLTSVM()
# classifier.fit(x_train, y_train)
# prediction = classifier.predict(x_val)

# acc=accuracy_score(y_val,predictions)

# ham=hamming_loss(y_val, predictions)

# print("Accuracy:",acc,"Hamming Loss:",ham)

### Decision Tree

In [59]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(x_res2, y_res2)
prediction = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### Extra Tree

In [60]:
from sklearn.tree import ExtraTreeClassifier

classifier = ExtraTreeClassifier()
classifier.fit(x_res2, y_res2)
prediction = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### Extra Tree Ensemble

In [61]:
from sklearn.ensemble import ExtraTreesClassifier

classifier = ExtraTreesClassifier()
classifier.fit(x_res2, y_res2)
prediction = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

/home/rahulsuresh/python-environments/env/lib/python3.5/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### MLP -> Not working due to label issue

In [43]:
# from sklearn.neural_network import MLPClassifier

# classifier = MLPClassifier()
# classifier.fit(x_train, y_train)
# prediction = classifier.predict(x_val)

# acc=accuracy_score(y_val,predictions)

# ham=hamming_loss(y_val, predictions)

# print("Accuracy:",acc,"Hamming Loss:",ham)

### Radius Neighbors Classifier -> Not Working because of absence of neigbours for many samples 

In [ ]:
# from sklearn.neighbors import RadiusNeighborsClassifier

# classifier = RadiusNeighborsClassifier()
# classifier.fit(x_train, y_train)
# prediction = classifier.predict(x_val)

# acc=accuracy_score(y_val,predictions)

# ham=hamming_loss(y_val, predictions)

# print("Accuracy:",acc,"Hamming Loss:",ham)

### Random Forest

In [62]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(x_res2, y_res2)
prediction = classifier.predict(x_val)

acc=accuracy_score(y_val,predictions)

ham=hamming_loss(y_val, predictions)

print("Accuracy:",acc,"Hamming Loss:",ham)

/home/rahulsuresh/python-environments/env/lib/python3.5/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.0005 Hamming Loss: 0.09932954545454545


### Ridge Classifier -> Not working due to label issue

In [46]:
# from sklearn.linear_model import RidgeClassifierCV

# classifier = RidgeClassifierCV()
# classifier.fit(x_train, y_train)
# prediction = classifier.predict(x_val)

# acc=accuracy_score(y_val,predictions)

# ham=hamming_loss(y_val, predictions)

# print("Accuracy:",acc,"Hamming Loss:",ham)